In [1]:
import pandas as pd
import numpy as np

import gc; gc.enable()

import warnings
warnings.filterwarnings("ignore")

In [2]:
folder_path = '../input/'

In [3]:
train_identity = pd.read_csv(f'{folder_path}train_identity.csv')
train_transaction = pd.read_csv(f'{folder_path}train_transaction.csv')

In [4]:
# let's combine the data and work with the whole dataset
train = pd.merge(train_transaction, 
                 train_identity, 
                 on='TransactionID', 
                 how='left')

del train_transaction; del train_identity; gc.collect()

39

In [5]:
# # idiot-test
# train = train.head(100).copy()
# gc.collect()

target = "isFraud"
_id = "TransactionID"

used_cols = [col for col in train.columns.tolist() if col not in [target, _id]]
# used_cols = ["TransactionDT", "TransactionAmt"]
keep_cols = used_cols + [target]

train = train[keep_cols].copy()
gc.collect()

7

In [6]:
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string


max_bin = 20
force_bin = 3


# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.sum().EVENT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.EVENT_RATE/d3.NON_EVENT_RATE)
    d3["IV"] = (d3.EVENT_RATE-d3.NON_EVENT_RATE)*np.log(d3.EVENT_RATE/d3.NON_EVENT_RATE)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    df2 = df1.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.groups
    d3["MAX_VALUE"] = df2.groups
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y     
    d3["EVENT_RATE"] = d3.EVENT/d3.sum().EVENT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.EVENT_RATE/d3.NON_EVENT_RATE)
    d3["IV"] = (d3.EVENT_RATE-d3.NON_EVENT_RATE)*np.log(d3.EVENT_RATE/d3.NON_EVENT_RATE)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    
    iv['STRENGTH'] = 'useless'
    iv.loc[iv['IV'] >= 0.02, 'STRENGTH'] = 'weak'
    iv.loc[iv['IV'] >= 0.1, 'STRENGTH'] = 'medium'
    iv.loc[iv['IV'] >= 0.3, 'STRENGTH'] = 'strong'
    iv.loc[iv['IV'] > 0.5, 'STRENGTH'] = 'suspicious'
    
    iv = iv.reset_index()
    
    return(iv_df,iv)


In [7]:
import pandas as pd
import numpy as np

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.externals import joblib 


class BaseTransformer(BaseEstimator, TransformerMixin):
    """
    Base class.
    """
    def __init__(self, columns=None, suffix='_base'):
        self.columns = columns
        self.suffix = suffix
        self.scaler = None
    
    def fit(self, X, y=None):
        if self.columns is None:
            self.columns = X.columns.tolist()
        self.scaler.fit(X[self.columns])
    
    def transform(self, X, y=None):
        return None
    
    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)
    
    def save(self, path):
        joblib.dump((self.columns, self.suffix, self.scaler), path)
    
    def load(self, path):
        self.columns, self.suffix, self.scaler = joblib.load(path)

/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [8]:
class WoeEncoder(BaseTransformer):
    """
    Weight of Evidence Encoding.
    """
    def __init__(self, columns=None, suffix="_woe"):
        self.columns = columns
        self.suffix = suffix
        self.iv_df = pd.DataFrame()
        self.iv = pd.DataFrame()
        
    def fit(self, df, y=None):
        if self.columns is None:
            self.columns = df.columns.tolist()
        
        iv_df, iv = data_vars(df[self.columns], y)
        
        self.iv_df = iv_df
        self.iv = iv
    
    def transform(self, df):
        for col in self.columns:
            LBs = self.iv_df[self.iv_df['VAR_NAME'] == col]['MIN_VALUE'].tolist()
            UBs = self.iv_df[self.iv_df['VAR_NAME'] == col]['MAX_VALUE'].tolist()
            
            criteria = [df[col].between(LBs[i], UBs[i]) for i in range(len(LBs))]
            values = self.iv_df[self.iv_df['VAR_NAME'] == col]['WOE'].tolist()
            default = self.iv_df[self.iv_df['VAR_NAME'] == col][self.iv_df['MIN_VALUE'].isna()]['WOE'].mean()

            df[col + self.suffix] = np.select(criteria, values, default)

            del LBs; del UBs; del criteria; del values; del default; gc.collect()
            
        return df
    
    def fit_transform(self, df, y):
        self.fit(df, y)
        return self.transform(df)

In [9]:
# WoE encode all of the things
woe = WoeEncoder(columns=used_cols, suffix='')

In [10]:
y = train[target]
# y = [0]*70 + [1]*30

train = woe.fit_transform(train, y)

In [11]:
woe.iv[woe.iv.STRENGTH == 'strong']

,VAR_NAME,IV,STRENGTH
17,D12,0.456384,strong
18,D13,0.359362,strong
19,D14,0.424995,strong
21,D2,0.355872,strong
22,D3,0.329245,strong
24,D5,0.423862,strong
25,D6,0.401412,strong
26,D7,0.464065,strong
28,D9,0.397365,strong
34,M4,0.342531,strong


In [12]:
woe.iv[woe.iv.STRENGTH == 'medium']

,VAR_NAME,IV,STRENGTH
0,C1,0.158372,medium
2,C11,0.166970,medium
6,C2,0.193810,medium
9,C5,0.163891,medium
14,D1,0.119214,medium
15,D10,0.281103,medium
16,D11,0.281640,medium
20,D15,0.197838,medium
23,D4,0.112419,medium
40,P_emaildomain,0.183120,medium


In [13]:
train

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,...,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,isFraud
0,-0.044803,-0.037092,-0.554564,-0.026764,0.315451,-0.353155,0.836495,0.167959,0.679113,-0.315153,-0.368065,-0.465464,-0.137818,NaN,NaN,-0.356375,-0.426897,0.000513,-0.285993,0.132445,-0.222418,-0.22079,-0.331708,0.145983,-0.314006,0.148828,-0.266679,0.269645,0.107007,-0.413075,0.274972,-1.058534,0.032027,-0.106254,-0.345600,-0.268909,-0.350819,-0.350819,0.351131,-0.330380,...,-0.525153,-0.312178,-0.312181,-0.508570,-0.500976,-0.012499,-0.012499,-0.350819,-0.350898,-0.524479,NaN,-0.489079,-0.046126,NaN,NaN,-0.509692,-0.123996,-0.508958,-0.508344,-0.012492,-0.012474,NaN,-0.011993,-0.012079,-0.012434,NaN,NaN,NaN,NaN,NaN,-0.043373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,-0.044803,-0.037092,-0.554564,0.128430,0.050556,-0.353155,-0.020411,0.167959,0.679113,-0.315153,-0.368065,0.265707,-0.137818,0.197246,NaN,-0.356375,-0.426897,0.000513,-0.285993,0.132445,-0.222418,-0.22079,-0.331708,0.145983,-0.314006,-0.324186,-0.266679,0.269645,0.107007,0.292104,0.274972,0.190193,0.317010,-0.106254,-0.345600,-0.268909,-0.350819,-0.350819,0.351131,0.416437,...,-0.525153,-0.312178,-0.312181,-0.508570,-0.500976,-0.012499,-0.012499,-0.350819,-0.350898,-0.524479,NaN,-0.489079,-0.046126,NaN,NaN,-0.509692,-0.123996,-0.508958,-0.508344,-0.012492,-0.012474,NaN,-0.011993,-0.012079,-0.012434,NaN,NaN,NaN,NaN,NaN,-0.043373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,-0.044803,-0.037092,-0.554564,0.128430,0.050556,-0.353155,-0.007663,0.167959,-0.377988,-0.315153,-0.368065,-0.465464,-0.137818,1.027855,NaN,-0.356375,-0.426897,0.000513,-0.285993,0.132445,-0.222418,-0.22079,-0.331708,0.145983,-0.314006,-0.324186,-0.266679,0.269645,0.107007,0.292104,0.274972,0.190193,0.317010,-0.106254,-0.345600,-0.268909,-0.350819,-0.350819,0.351131,-0.951759,...,-0.525153,-0.312178,-0.312181,-0.508570,-0.500976,-0.012499,-0.012499,-0.350819,-0.350898,-0.524479,NaN,-0.489079,-0.046126,NaN,NaN,-0.509692,-0.123996,-0.508958,-0.508344,-0.012492,-0.012474,NaN,-0.011993,-0.012079,-0.012434,NaN,NaN,NaN,NaN,NaN,-0.043373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,-0.044803,-0.037092,-0.554564,-0.109230,0.050556,-0.353155,-0.020411,0.167959,-0.377988,-0.315153,-0.368065,0.265707,-0.137818,-0.473082,NaN,-0.149298,0.547204,0.000513,-0.285993,0.132445,0.494151,-0.22079,-0.331708,0.145983,-0.314006,-0.324186,-0.266679,-0.497488,0.107007,-0.413075,-0.758347,0.605765,-0.497475,0.935506,-0.345600,-0.268909,-0.350819,-0.350819,-0.795461,0.416437,...,-0.525153,-0.312178,-0.312181,-0.508570,-0.500976,-0.012499,-0.012499,-0.350819,-0.350898,-0.524479,NaN,-0.489079,-0.046126,NaN,NaN,-0.509692,-0.123996,-0.508958,-0.508344,-0.012492,-0.012474,NaN,-0.011993,-0.012079,-0.012434,NaN,NaN,NaN,NaN,NaN,-0.043373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,-0.044803,-0.037092,0.322297,0.128430,0.050556,-0.353155,-0.020411,0.167959,0.679113,-0.315153,-0.368065,0.265707,-0.137818,0.197246,NaN,-0.356375,-0.426897,0.000513,-0.285993,0.132445,-0.222418,-0.22079,-0.331708,0.145983,-0.314006,-0.324186,-0.266679,0.269645,0.107007,0.292104,0.274972,0.190193,0.032027,-0.106254,-0.345600,-0.268909,-0.350819,-0.350819,0.395903,0.416437,...,0.746241,-0.312178,-0.312181,-0.508570,-0.500976,-0.012499,-0.012499,-0.350819,-0.350898,0.868909,0.043613,-0.489079,0.031677,-0.512644,-0.519056,0.225316,-0.123996,0.720782,0.684511,-0.012492,-0.012474,NaN,-0.011993,-0.012079,-0.012434,NaN,-0.449254,-0.468447,0.278804,-0.128817,0.678983,0.157733,-0.136716,-0.614605,0.030827,0.049662,-0.311869,0.268660,0.000000,0
5,-0.044803,-0.037092,-0.554564,0.128430,0.050556,-0.353155,-0.007663,-0.104641,-0.377988,-0.

In [14]:
# create LogReg model

In [15]:
# visualize confusion matrices

In [16]:
# load in test data

In [17]:
# encode test data

In [18]:
# predict

In [19]:
# save